<a href="https://colab.research.google.com/github/sakshamchecker/USAirlineTweetML/blob/main/USAirlineTweetAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd 
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize

In [13]:
 import nltk
 nltk.download('punkt')
 nltk.download('stopwords')
 nltk.download('state_union')
 nltk.download('averaged_perceptron_tagger')
 nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Unzipping corpora/state_union.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
dataset=pd.read_csv("training_twitter_x_y_train.csv")

In [4]:
dataset.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [20]:
tweets=[]
len(dataset['text'][0].split())

24

In [14]:
word_tokenize(dataset['text'][0])

['@',
 'SouthwestAir',
 'I',
 'am',
 'scheduled',
 'for',
 'the',
 'morning',
 ',',
 '2',
 'days',
 'after',
 'the',
 'fact',
 ',',
 'yes..not',
 'sure',
 'why',
 'my',
 'evening',
 'flight',
 'was',
 'the',
 'only',
 'one',
 'Cancelled',
 'Flightled']

In [10]:
dataset.shape[0]

10980

In [21]:
for i in range(dataset.shape[0]):
  tweets.append((word_tokenize(dataset['text'][i]), dataset['airline_sentiment'][i]))

In [22]:
tweets[0]

(['@',
  'SouthwestAir',
  'I',
  'am',
  'scheduled',
  'for',
  'the',
  'morning',
  ',',
  '2',
  'days',
  'after',
  'the',
  'fact',
  ',',
  'yes..not',
  'sure',
  'why',
  'my',
  'evening',
  'flight',
  'was',
  'the',
  'only',
  'one',
  'Cancelled',
  'Flightled'],
 'negative')

In [27]:
 from nltk import pos_tag

In [29]:
from nltk.stem import WordNetLemmatizer
lemmetizer=WordNetLemmatizer()

In [23]:
from nltk.corpus import stopwords
stops=set(stopwords.words('english'))
import string
stops.update(list(string.punctuation))

In [24]:
from nltk.corpus import wordnet

def get_simple_pos(tag):
  if tag.startswith('J'):
    return wordnet.ADJ
  if tag.startswith('V'):
    return wordnet.VERB
  if tag.startswith('N'):
    return wordnet.NOUN
  if tag.startswith('R'):
    return wordnet.ADV
  return wordnet.NOUN

In [25]:
def clean(words):
  output_words=[]
  for w in words:
    if w.lower() not in stops:
      pos=pos_tag([w])
      clean_word=lemmetizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
      output_words.append(clean_word.lower())
  return output_words

In [30]:
tweets=[(clean(doc),category) for doc, category in tweets]

In [31]:
tweets[:5]

[(['southwestair',
   'schedule',
   'morning',
   '2',
   'day',
   'fact',
   'yes..not',
   'sure',
   'even',
   'flight',
   'one',
   'cancelled',
   'flightled'],
  'negative'),
 (['southwestair',
   'see',
   'worker',
   'time',
   'time',
   'go',
   'beyond',
   'love',
   'fly',
   'guy',
   'thank'],
  'positive'),
 (['united',
   'flew',
   'ord',
   'miami',
   'back',
   'great',
   'crew',
   'service',
   'leg',
   'thanks'],
  'positive'),
 (['southwestair', 'dultch97', "'s", 'horse', 'radish', '😤🐴'], 'negative'),
 (['united',
   'flight',
   'ord',
   'delayed',
   'air',
   'force',
   'one',
   'last',
   'flight',
   'sbn',
   '8:20',
   '5',
   'min',
   'land'],
  'negative')]

In [32]:
len(tweets)

10980

In [33]:
training_tweets=tweets[:7500]
testing_tweets=tweets[7500:]

In [34]:
all_words=[]
for twee in training_tweets:
  all_words+=twee[0]

In [36]:
# all_words

In [37]:
freq=nltk.FreqDist(all_words)
common=freq.most_common(3000)

features=[i[0] for i in common]

In [38]:
def get_feature_dict(words):
  current_features={}
  words_set=set(words)
  for w in features:
    current_features[w]=w in words_set
  return current_features

In [39]:
training_data=[(get_feature_dict(doc),category) for doc, category in training_tweets]

In [40]:
testing_data=[(get_feature_dict(doc),category) for doc, category in testing_tweets]

In [41]:
from nltk import NaiveBayesClassifier

In [42]:
classifier=NaiveBayesClassifier.train(training_data)

In [44]:
nltk.classify.accuracy(classifier, testing_data)

0.7793103448275862